In [10]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [11]:
company_name = "GES - UHE-ESTREITO"

In [12]:
company = Company.objects.get(name=company_name)

In [13]:
from tqdm.notebook import tqdm
from openpyxl import load_workbook
from datetime import datetime
import json
from helpers.strings import get_obj_from_path
from helpers.signals import DisableSignals
from django.contrib.gis.geos import Point
from django.contrib.gis.geos import GeometryCollection
from django.contrib.admin.utils import flatten

In [14]:
file_name = "2023_Planilha_Geral_PBA_07 retrabalho"

In [15]:
def transform_coordinates(x, y, srid=31982):

    pnt = Point(x, y, srid=srid)
    pnt.transform(4326)
    
    lng = pnt.coords[1]
    lat = pnt.coords[0]
    
    return Point(lat, lng)

In [16]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [17]:
def get_sheet_names(filename):
    wb = load_workbook(filename + ".xlsx")
    return wb.sheetnames

In [18]:
def load_sheet(filename, sheetname):
    
    wb = load_workbook(filename=filename+'.xlsx')
    sheet = wb[sheetname]
    data = []

    headers = [cell.value for cell in sheet[1]]

    for row in sheet.iter_rows(min_row=2):
        row_dict = {}
        for key, cell in zip(headers, row):
            row_dict[key] = cell.value

        data.append(row_dict)
    ######
#     wb = load_workbook(filename + ".xlsx")
#     ws = wb[sheetname]
    
#     header = []
#     values = []

#     for index, row in enumerate(ws.rows):
#         if index == 0:
#             continue
#         elif index == 1:
#             header = list([a.value for a in row])
#             continue
#         obj = {}
#         for col_index, cell in enumerate(row):
#             value = cell.value
#             obj[header[col_index]] = value
#         values.append(obj)
        
    return data

In [19]:
operational_control = OperationalControl.objects.get(
    firm__company=company,
    kind="710"
)
status = ServiceOrderActionStatus.objects.get(
    companies=company, 
    name="Registro Deferido"
)
firm = Firm.objects.get(
    company=company, 
    name="CESTE Meio Ambiente"
)
occ = OccurrenceType.objects.get(
    uuid='21443c13-7f29-4701-9662-709c56b503db'
)
user=User.objects.get(
    username="engie"
)
operational_control,status,firm,occ,user

(<OperationalControl: [GES - UHE-ESTREITO]: 710 - [Serviços de desenvolvimento de plataforma para gestão de equipes e serviços ambientais]>,
 <ServiceOrderActionStatus: Registro Deferido - OCCURRENCE_RECORD_STATUS>,
 <Firm: [42ddc389-3258-4daf-868a-097235427e0f: GES - UHE-ESTREITO] 2aadce83-4be0-4b4b-aaba-be8dee8be318: CESTE Meio Ambiente>,
 <OccurrenceType: Vistorias Residenciais PBA07 - ['GES - UHE-ESTREITO']>,
 <User: engie - Suporte Kartado>)

In [29]:
print(list(OccurrenceRecord.objects.filter(operational_control=operational_control,
                                company=company).values_list('city__name').annotate(qtd_por_cidade=Count('uuid'))))

[('Babaçulândia', 483), ('Carolina', 1485), ('Filadélfia', 355), ('Palmeirante', 396)]


##### Essa linha abaixo é para adicionar opções caso não existam em algum selectOptions

In [11]:
# for form in OccurrenceType.objects.filter(company=company, occurrence_kind="701"):
#     position = next(a for a in form.form_fields['fields'] if a['apiName'] == "operationalPosition")
#     position['selectOptions']['options'].append({"name": "Dique 1","value": "9"})
#     position['selectOptions']['options'].append({"name": "Dique 2","value": "10"})
#     position['selectOptions']['options'].append({"name": "Dique 3","value": "11"})
#     form.save()

In [12]:
# antigo jeito de pegar os dados da planilha

# wb = load_workbook(filename=file_name+'.xlsx')
# sheet = wb.active
# data = []

# headers = [cell.value for cell in sheet[1]]

# for row in sheet.iter_rows(min_row=2):
#     row_dict = {}
#     for key, cell in zip(headers, row):
#         row_dict[key] = cell.value

#     data.append(row_dict)
# data[0]

In [20]:
sheets = get_sheet_names(file_name)
recs = []

for sheet in sheets:
    form = OccurrenceType.objects.filter(company=company, name='Vistorias Residenciais PBA07')
    print("running", sheet)
    
    if not form.exists():
        print("skipping", sheet)
        continue
    
    form = form.first()

#     values = load_sheet(file_name, sheet)
    data = load_sheet(file_name, sheet)
    for record in tqdm(data):
        
        form_data = {
            "token": record["FICHA"] if record.get("FICHA") is not None else "",
            "inspection_date": record["DATA DA VISTORIA"].strftime('%Y-%m-%dT03:00:00.000Z') if record.get("DATA DA VISTORIA") is not None and type(record["DATA DA VISTORIA"]) == datetime else "",
            "ceste_request": record["SOLICITAÇÃO CESTE"] if record.get("SOLICITAÇÃO CESTE") is not None else "",
            "inspection_na": record["NA"] if record.get("NA") is not None else "",
            "owner": record["PROPRIETÁRIO"] if record.get('PROPRIETÁRIO') is not None else "",
            "cpf": record["CPF"] if record.get("CPF") is not None else "",
            "phone": record["TELEFONE"] if record.get('TELEFONE') is not None else "",
            "address": record["ENDEREÇO"] if record.get('ENDEREÇO') is not None else "",
            "address_number": record["Nº"] if record.get('Nº') is not None else "",
            "district": record["BAIRRO"] if record.get('BAIRRO') is not None else "",
            "intervention": get_value('Intervenção', occ, record["INTERVENÇÃO"].strip().capitalize() if record["INTERVENÇÃO"] != "RUC" else record["INTERVENÇÃO"]) if record.get('INTERVENÇÃO') is not None else None,
            "negotiated": record["NEGOCIADO"] if record.get('NEGOCIADO') is not None else "",
            "class_vis": record["CLASSE"] if record.get('CLASSE') is not None else "",
            "value_vis": record["VALOR"] if record.get('VALOR') is not None else "",
            "date_negotiation": record["DATA NEGOCIAÇÃO"].strftime('%Y-%m-%dT03:00:00.000Z') if record.get('DATA NEGOCIAÇÃO') is not None and type(record["DATA NEGOCIAÇÃO"]) == datetime else "",
            "date_delivered_check": record["DATA ENTREGA CHEQUES 1º"].strftime('%Y-%m-%dT03:00:00.000Z') if record.get('DATA ENTREGA CHEQUES 1º') is not None and type(record["DATA ENTREGA CHEQUES 1º"]) == datetime else "",
            "date_delivered_check2": record["DATA ENTREGA CHEQUES 2º"].strftime('%Y-%m-%dT03:00:00.000Z') if record.get("DATA ENTREGA CHEQUES 2º") is not None and type(record["DATA ENTREGA CHEQUES 2º"]) == datetime else "",
            "date_delivered_check3": record["DATA ENTREGA CHEQUES 3º"].strftime('%Y-%m-%dT03:00:00.000Z') if record.get("DATA ENTREGA CHEQUES 3º") is not None and type(record["DATA ENTREGA CHEQUES 3º"]) == datetime else "",
            "date_delivered_check4": record["DATA ENTREGA CHEQUES 4º"].strftime('%Y-%m-%dT03:00:00.000Z') if record.get("DATA ENTREGA CHEQUES 4º") is not None and type(record["DATA ENTREGA CHEQUES 4º"]) == datetime else "",
            "step2017": record["Etapas 2017"] if record.get('Etapas 2017') is not None else "",
            "step2018": record["Etapas 2018"] if record.get('Etapas 2018') is not None else "",
            "received_non_intervention": record["RECEBEU O COMUNICADO DA NÃO INTERVENÇÃO"] if record.get('RECEBEU O COMUNICADO DA NÃO INTERVENÇÃO') is not None else "",
            "comunication_method": record["METODO DE COMUNICAÇÃO DA NÃO INTERVENÇÃO (CARTA, REUNIÃO, TELEFONEMA"] if record.get('METODO DE COMUNICAÇÃO DA NÃO INTERVENÇÃO (CARTA, REUNIÃO, TELEFONEMA') is not None else "",
            "date_non_intervention": record["DATA DA COMUNICAÇÃO DO NÃO INTERFERIDO"].strftime('%Y-%m-%dT03:00:00.000Z') if record.get('DATA DA COMUNICAÇÃO DO NÃO INTERFERIDO') is not None and type(record.get('DATA DA COMUNICAÇÃO DO NÃO INTERFERIDO')) == datetime else "",
            "number_register_non_intervention": record["NÚMERO DO REGISTRO DE COMUNICAÇÃO DOS NÃO INTERFERIDOS"] if record.get('NÚMERO DO REGISTRO DE COMUNICAÇÃO DOS NÃO INTERFERIDOS') is not None else "",
            "notes_consultant": record["OBSERVAÇÕES CONSULTORES"] if record.get('OBSERVAÇÕES CONSULTORES') is not None else "",
            "notes_ceste": record["OBSERVAÇÕES CESTE"] if record.get('OBSERVAÇÕES CESTE') is not None else "",
            "x_sirgas": record['X(SIRGAS 2000)'] if record['X(SIRGAS 2000)'] is not None else "",
            "y_sirgas": record['Y(SIRGAS 2000)'] if record['Y(SIRGAS 2000)'] is not None else ""
        }
        p=transform_coordinates(float(record['X(SIRGAS 2000)']), float(record['Y(SIRGAS 2000)']),srid=31983) if record['X(SIRGAS 2000)'] is not None else None
        gc = GeometryCollection()
        gc.append(p) if record['X(SIRGAS 2000)'] is not None else None
        rec = OccurrenceRecord(
            company = company,
            created_by = user,
            city=City.objects.filter(monitoringpoint__monitoring_plan__company=company,
                                      name__icontains=sheet.split('Vistorias_')[1].strip()).first(),
            uf_code = City.objects.filter(monitoringpoint__monitoring_plan__company=company,
                                      name__icontains=sheet.split('Vistorias_')[1].strip()).first().uf_code,
            point=p,
            geometry=gc,
            occurrence_type = occ,
            status = status,
            form_data = form_data,
            firm = firm,
            operational_control = operational_control,
            datetime = datetime.now()
        )
        recs.append(rec)

running Vistorias_Carolina


  0%|          | 0/1501 [00:00<?, ?it/s]

running Vistorias_Babaçulândia 


  0%|          | 0/599 [00:00<?, ?it/s]

running Vistorias_Filadélfia


  0%|          | 0/355 [00:00<?, ?it/s]

running Vistorias_Palmeirante


  0%|          | 0/396 [00:00<?, ?it/s]

In [21]:
recs[1500].__dict__

{'_state': <django.db.models.base.ModelState at 0x7fdca0729890>,
 'uuid': UUID('e2c612ab-493d-4278-9e05-9b6ccdb57e76'),
 'datetime': datetime.datetime(2024, 2, 2, 10, 58, 54, 136177),
 'number': '',
 'created_at': None,
 'updated_at': None,
 'reviews': 0,
 'company_id': UUID('42ddc389-3258-4daf-868a-097235427e0f'),
 'editable': True,
 'is_approved': False,
 'uf_code': 21,
 'city_id': UUID('5238f4ca-08fa-41c9-b1d2-ffccc4dc3d43'),
 'location_id': None,
 'place_on_dam': '',
 'river_id': None,
 'point': None,
 'geometry': <GeometryCollection object at 0x7fdca070e520>,
 'properties': [],
 'distance_from_dam': 0,
 'other_reference': '',
 'origin': None,
 'origin_media': None,
 'informer': {},
 'territorial_administration': '',
 'created_by_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'),
 'search_tag_description': '',
 'occurrence_type_id': UUID('21443c13-7f29-4701-9662-709c56b503db'),
 'form_data': {'token': '',
  'inspection_date': '',
  'ceste_request': '',
  'inspection_na': '',
  'own

In [14]:
for i,a in enumerate(recs):
    for b in a.form_data.values():
        if type(b) == datetime:
            print(i)

2254


In [16]:
recs[2254].form_data['number_register_non_intervention']=''

In [26]:
sheet.split('Vistorias_')[1]

'Filadelfia'

In [17]:
OccurrenceRecord.objects.bulk_create(recs)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:30.257678) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:30.273392) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:30.288062) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.p

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:34.660157) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:34.674538) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:34.688642) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.p

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:40.317109) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:40.331452) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:40.345960) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.p

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:46.028941) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:46.046016) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:46.061969) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.p

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:46.975891) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:46.989853) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:09:47.004211) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.p

...

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:01.758774) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:01.776437) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:01.801391) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.p

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:10.917584) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:10.936438) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:10.951665) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.p

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:19.631813) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:19.646196) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:19.665335) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.p

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:25.290987) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:25.305343) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-01-26 22:10:25.319222) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.p

[<OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.257678>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.273392>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.288062>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.305976>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.320845>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.335023>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.348803>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.363323>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.377505>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.392359>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.411718>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.427081>,
 <OccurrenceRecord: [GES - UHE-ESTREITO] -  - 2024-01-26 22:09:30.442134>,
 <OccurrenceRecord: [GES 

### !!!!! Apagar tudo antes de importar?

In [21]:
# with DisableSignals():
#     print(OccurrenceRecord.objects.filter(company=company,operational_control=operational_control).count())
#     OccurrenceRecord.objects.filter(company=company,operational_control=operational_control).delete()

2473


In [1]:
OccurrenceRecord._meta.get_fields()

(<ManyToOneRel: occurrence_records.historicaloccurrencerecord>,
 <ManyToOneRel: occurrence_records.occurrencerecordwatcher>,
 <ManyToManyRel: occurrence_records.customdashboard>,
 <ManyToOneRel: occurrence_records.dataseries>,
 <ManyToOneRel: service_orders.serviceorderaction>,
 <ManyToManyRel: service_orders.procedure>,
 <ManyToOneRel: monitorings.monitoringcollect>,
 <ManyToOneRel: monitorings.materialusage>,
 <django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.DateTimeField: datetime>,
 <django.db.models.fields.CharField: number>,
 <django.db.models.fields.DateTimeField: created_at>,
 <django.db.models.fields.DateTimeField: updated_at>,
 <django.db.models.fields.IntegerField: reviews>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.db.models.fields.BooleanField: editable>,
 <django.db.models.fields.BooleanField: is_approved>,
 <django.db.models.fields.IntegerField: uf_code>,
 <django.db.models.fields.related.ForeignKey: city>,
 <django.db.models.fi

In [95]:
for record in data:
    for k,v in record.items():
        if type(v) is datetime:
            if v.month or v.day:
                continue
            else:
                record[k] = datetime(v,1,1)
    if type(record['Data Negociação']) is int:
        record['Data Negociação'] = datetime(record['Data Negociação'],1,1)

In [96]:
data[1438]

{'MUNICÍPIO': 'BABAÇULÂNDIA',
 'CONTAGEM': 1439,
 'Ficha': 17,
 'Data da Vistoria': datetime.datetime(2016, 3, 2, 0, 0),
 'Solicitação Ceste': 183,
 'NA': 1.1,
 'Proprietário': 'POLIANA RODRIGUES VIANA MACHADO',
 'CPF': '861.141.311-34',
 'Telefone': '(63) 8462-7167',
 'Endereço': 'RUA VS 2A, QD. 07, LT 29',
 'Nº': 'S/N',
 'Bairro': 'NOVO MILÊNIO I',
 'X(SIRGAS 2000)': 195192.040675,
 'Y(SIRGAS 2000)': 9202559.4092,
 'Intervenção': 'SIM',
 'Negociado': 'SIM',
 'Classe': 'E',
 'Valor': 11800,
 'Data Negociação': datetime.datetime(2016, 1, 1, 0, 0),
 'Data Entrega Cheques': datetime.datetime(2017, 1, 24, 0, 0),
 'Etapas 2017': None,
 'Etapas 2018': None,
 'Recebeu o Comunicado da Não Intervenção': 'NÃO APLICÁVEL',
 'Método de Comunicação da Não Intervenção': 'NÃO APLICÁVEL',
 'Data da Comunicação do Não Interferido': 'NÃO APLICÁVEL',
 'Número do registro de comunicação dos Não Interferidos': 'NÃO APLICÁVEL',
 'Observações Consultores': '',
 'Observações CESTE': None}

# Alteração Itens Importados

In [14]:
ocr=OccurrenceRecord.objects.filter(company=company,operational_control=operational_control,form_data__).count()

In [18]:
for i,a in enumerate(tqdm(data)):
    try:
        ocr=OccurrenceRecord.objects.get(company=company,
                                         operational_control=operational_control,
                                         form_data__token=a.get('Ficha'),
                                         form_data__owner=a.get('Proprietário'))
    except:
        print(i+2,a.get('Ficha'),a.get('Proprietário'))

  0%|          | 0/2473 [00:00<?, ?it/s]

95 94 EDAILTON PEREIRA DOS SANTOS
104 103 RAIMUNDO MACEDO BARROS
109 108 MARIA DE SOUSA MARANHÃO
168 167 ROSA OLIVEIRA DE SOUSA
169 167 ROSA OLIVEIRA DE SOUSA
170 167 ROSA OLIVEIRA DE SOUSA
171 167 ROSA OLIVEIRA DE SOUSA
192 188 RAIMUNDO BARROS DE SOUSA
196 192 DOURIVAL TEIXEIRA DE SOUSA
197 192 DOURIVAL TEIXEIRA DE SOUSA
213 208 MARIA DINÁ FELIX DA SILVA
215 210 MANOEL PINHEIRO NOLETO
218 213 ADEVALDO PEREIRA DE BRITO
270 265 MARIA DO SOCORRO CUNHA BARBOSA
271 265 MARIA DO SOCORRO CUNHA BARBOSA
296 290 UBIRAJARA BARBOSA JUCÁ
297 290 UBIRAJARA BARBOSA JUCÁ
351 344 MARIA ESPINOLA NOLETO
369 363 ANTONIA LIMA DE SOUSA
581 575 BELTO BARBOSA DA SILVA
585 579 ROSILDA CUNHA MENEZES
878 94 EDAILTON PEREIRA DOS SANTOS
880 103 RAIMUNDO MACEDO BARROS
881 108 MARIA DE SOUSA MARANHÃO
885 188 RAIMUNDO BARROS DE SOUSA
888 208 MARIA DINÁ FELIX DA SILVA
889 210 MANOEL PINHEIRO NOLETO
891 213 ADEVALDO PEREIRA DE BRITO
895 344 MARIA ESPINOLA NOLETO
896 363 ANTONIA LIMA DE SOUSA
900 575 BELTO BARBOSA DA S

In [ ]:
point=transform_coordinates(float(record['X(SIRGAS 2000)']), float(record['Y(SIRGAS 2000)']),srid=31983) if record['X(SIRGAS 2000)'] is not None else None,
geometry=GeometryCollection(),
geometry.append(point),

In [14]:
OccurrenceRecord.objects.filter(number__in=['UHMA-RG-2024.0062','UHPF-RG-2024.0006']).delete()

(12,
 {'occurrence_records.OccurrenceRecord_search_tags': 8,
  'files.File': 2,
  'occurrence_records.OccurrenceRecord': 2})